## Deliverable 2. Create a Customer Travel Destinations Map.

In [33]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [34]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.6500,115.3333,68.09,88,18,15.59,few clouds
1,1,Rikitea,PF,-23.1203,-134.9692,77.27,75,82,12.50,broken clouds
2,2,Kavieng,PG,-2.5744,150.7967,81.41,75,87,4.47,overcast clouds
3,3,Mahon,ES,39.8885,4.2658,52.36,82,20,9.22,few clouds
4,4,Ushuaia,AR,-54.8000,-68.3000,60.08,68,100,3.83,overcast clouds


In [35]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 70


In [36]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.6500,115.3333,68.09,88,18,15.59,few clouds
3,3,Mahon,ES,39.8885,4.2658,52.36,82,20,9.22,few clouds
4,4,Ushuaia,AR,-54.8000,-68.3000,60.08,68,100,3.83,overcast clouds
5,5,Punta Arenas,CL,-53.1500,-70.9167,57.11,58,100,9.64,overcast clouds
8,8,Kapaa,US,22.0752,-159.3190,69.48,97,31,5.01,light rain


In [37]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                164
City                   164
Country                160
Lat                    164
Lng                    164
Max Temp               164
Humidity               164
Cloudiness             164
Wind Speed             164
Current Description    164
dtype: int64

In [49]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.6500,115.3333,68.09,88,18,15.59,few clouds
3,3,Mahon,ES,39.8885,4.2658,52.36,82,20,9.22,few clouds
4,4,Ushuaia,AR,-54.8000,-68.3000,60.08,68,100,3.83,overcast clouds
5,5,Punta Arenas,CL,-53.1500,-70.9167,57.11,58,100,9.64,overcast clouds
8,8,Kapaa,US,22.0752,-159.3190,69.48,97,31,5.01,light rain
...,...,...,...,...,...,...,...,...,...,...
678,678,Hit,IQ,33.6416,42.8251,52.65,53,74,7.36,broken clouds
688,688,Nicolas Bravo,MX,18.4500,-88.9167,68.88,57,83,7.16,broken clouds
692,692,Ellenabad,IN,29.4500,74.6500,55.76,39,0,6.64,clear sky
702,702,Hirara,JP,24.8000,125.2833,68.67,84,100,22.59,overcast clouds


In [50]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Busselton,AU,68.09,few clouds,-33.6500,115.3333,
3,Mahon,ES,52.36,few clouds,39.8885,4.2658,
4,Ushuaia,AR,60.08,overcast clouds,-54.8000,-68.3000,
5,Punta Arenas,CL,57.11,overcast clouds,-53.1500,-70.9167,
8,Kapaa,US,69.48,light rain,22.0752,-159.3190,
12,Coihaique,CL,68.61,overcast clouds,-45.5752,-72.0662,
14,Mataura,NZ,56.23,moderate rain,-46.1927,168.8643,
15,Uvalde,US,58.41,clear sky,29.3669,-99.7670,
16,Victoria,HK,63.25,broken clouds,22.2855,114.1577,
18,Mount Gambier,AU,58.48,broken clouds,-37.8333,140.7667,


In [61]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

import numpy as np

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        hotel_df.loc[index, "Hotel Name"] = np.nan
                

In [77]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.dropna()
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Busselton,AU,68.09,few clouds,-33.6500,115.3333,Observatory Guest House
3,Mahon,ES,52.36,few clouds,39.8885,4.2658,ARTIEM Capri
4,Ushuaia,AR,60.08,overcast clouds,-54.8000,-68.3000,Albatross Hotel
5,Punta Arenas,CL,57.11,overcast clouds,-53.1500,-70.9167,Hotel Hain
8,Kapaa,US,69.48,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
...,...,...,...,...,...,...,...
674,Tolaga Bay,NZ,55.94,clear sky,-38.3667,178.3000,Tolaga Bay Motel
678,Hit,IQ,52.65,broken clouds,33.6416,42.8251,صالون زيرو
692,Ellenabad,IN,55.76,clear sky,29.4500,74.6500,Lovelysweets
702,Hirara,JP,68.67,overcast clouds,24.8000,125.2833,Hotel Atoll Emerald Miyakojima


In [65]:
# 8a. Create the output File (CSV)
output_data_file = '../Vacation_Search/WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [74]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt><strong>Hotel Name</strong></dt><dd>{Hotel Name}</dd>
<dt><strong>City</strong></dt><dd>{City}</dd>
<dt><strong>Country</strong></dt><dd>{Country}</dd>
<dt><strong>Current Weather</strong></dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [75]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))